# Pincecone first implementation

### Install libraries

In [ ]:
pip install pinecone-client

In [ ]:
pip install pinecone-datasets

In [ ]:
pip install pypdf

In [ ]:
pip install tiktoken

### Load data from pdf into Pinecone

In [1]:
# Load datas
from langchain.document_loaders import PyPDFLoader, TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

# Prompt
from langchain.prompts import PromptTemplate

# Vector database
from langchain.vectorstores import Pinecone
from langchain.embeddings.openai import OpenAIEmbeddings
import pinecone

# Llm
from langchain.chat_models import ChatOpenAI
from langchain.chains import RetrievalQA
from langchain.chains.conversation.memory import ConversationBufferWindowMemory
from langchain.schema import SystemMessage

#agents
from langchain.agents.types import AgentType
from langchain.agents import initialize_agent
from langchain.tools import Tool

# Environment
from dotenv import load_dotenv
import os

os.environ.clear()
load_dotenv()

c:\Users\guzma\anaconda3\Lib\site-packages\pinecone\index.py:4: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


True

### Load CV

In [4]:
#use your file 
loader = PyPDFLoader("../ruy.pdf")

data = loader.load()

print (f'You have {len(data)} document(s) in your data')
print (f'There are {len(data[0].page_content)} characters in your document')

You have 1 document(s) in your data
There are 3112 characters in your document


In [5]:
# Note: If you're using PyPDFLoader then we'll be splitting for the 2nd time.
# This is optional, test out on your own data.

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
texts = text_splitter.split_documents(data)
print (f'Now you have {len(texts)} documents')


Now you have 4 documents


### Create embedings and upload to pinecone

In [2]:
embeddings = OpenAIEmbeddings(openai_api_key=os.getenv('OPENAI_API_KEY'),chunk_size=1000)

In [3]:

# initialize pinecone
pinecone.init(
    api_key= os.getenv('PINECONE_API_KEY'),  # find at app.pinecone.io
    environment= os.getenv('PINECONE_ENVIRONMENT')   # next to api key in console
)
index = 'digital-twin'


In [10]:
docsearch = Pinecone.from_texts([t.page_content for t in texts], embeddings, index_name=index)


### Use data from vector database with llm

In [4]:
idx = pinecone.Index(index)
text_field = "text"

vectorstore = Pinecone(
    idx,
    embeddings,
    text_field
)

In [5]:
promptTopic = PromptTemplate(
    input_variables=["topic"],
    template="What experience does Ruy have related to {topic}? Give me an overview and a detailed list of technologies used in his {topic} experience",
    )

promptLanguage= PromptTemplate(
    input_variables=["language"],
    template="Is {language} within the programming lnguages Ruy can use? And show me an example of how has he used it",
    )

promptFramework = PromptTemplate(
    input_variables=["framework"],
    template="Does Ruy have any expierence using {framework}? Describe the experiences",
    )

In [6]:
# completion llm
llm = ChatOpenAI(
    openai_api_key=os.getenv('OPENAI_API_KEY'),
    model_name='gpt-3.5-turbo',
    temperature=1
)

conversational_memory = ConversationBufferWindowMemory(
    memory_key="chat_history", return_messages=True
)

qa = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=vectorstore.as_retriever(),
)

In [7]:
loader = TextLoader("./StudentPrompt.txt")

StudentPrompt = loader.load()[0].page_content

tools = [
    Tool(
        name="qa-student",
        func=qa.run,
        description="Answer questions related to the student",
    )
]

In [8]:
student = initialize_agent(
    agent=AgentType.CHAT_CONVERSATIONAL_REACT_DESCRIPTION,
    tools= tools,
    llm=llm,
    memory=conversational_memory,
    system_message=StudentPrompt,
    agent_kwargs={
        "system_message": StudentPrompt
    },
    verbose=True,
)

### Run the model

In [9]:
conversational_memory.load_memory_variables({})

{'chat_history': []}

In [10]:
student.run("Hi, Im a recruiter for a tech company. Can you briefly introduce yourself")



> Entering new AgentExecutor chain...
{
    "action": "qa-student",
    "action_input": "Can you briefly introduce yourself"
}
Observation: My name is Ruy Guzman Camacho. I am currently pursuing a B.S. in Computer Science and Technology at Instituto Tecnológico y de Estudios Superiores de Monterrey. I also have a programming technician degree from Universidad del Valle de Atemajac. I have experience as a Major League Hacking Production Engineering Fellow and as a volunteer student at Banco de alimentos de Guadalajara and IBM. I have worked on various projects as a full-stack developer, using languages such as Python, JavaScript, Ruby, and C++. I am proficient in frameworks like React, Node.js, Flask, and Flutter. I have knowledge of developer tools like Git, Docker, and Firebase.
Thought:{
    "action": "Final Answer",
    "action_input": "Sure! My name is Ruy Guzman Camacho. I am currently pursuing a B.S. in Computer Science and Technology at Instituto Tecnológico y de Estudios Supe

'Sure! My name is Ruy Guzman Camacho. I am currently pursuing a B.S. in Computer Science and Technology at Instituto Tecnológico y de Estudios Superiores de Monterrey. I also have a programming technician degree from Universidad del Valle de Atemajac. I have experience as a Major League Hacking Production Engineering Fellow and as a volunteer student at Banco de alimentos de Guadalajara and IBM. I have worked on various projects as a full-stack developer, using languages such as Python, JavaScript, Ruby, and C++. I am proficient in frameworks like React, Node.js, Flask, and Flutter. I have knowledge of developer tools like Git, Docker, and Firebase.'

In [11]:
student.run("I'm looking for a DevOps engineer, do you have any experience?")



> Entering new AgentExecutor chain...
{
  "action": "qa-student",
  "action_input": "Do you have any experience as a DevOps engineer?"
}
Observation: Based on the provided context, there is no explicit mention of experience as a DevOps engineer. The individual has experience in software development, full-stack development, production engineering, and has used tools and technologies related to DevOps such as Docker, CI/CD with GitHub Actions, and monitoring environments with Prometheus and Grafana. However, without specific information about working as a DevOps engineer, it cannot be concluded that they have direct experience in that role.
Thought:{
  "action": "Final Answer",
  "action_input": "While I don't have specific experience as a DevOps engineer, I have experience in software development and have worked with tools and technologies related to DevOps such as Docker, CI/CD with GitHub Actions, and monitoring environments with Prometheus and Grafana. I am confident in my ability 

"While I don't have specific experience as a DevOps engineer, I have experience in software development and have worked with tools and technologies related to DevOps such as Docker, CI/CD with GitHub Actions, and monitoring environments with Prometheus and Grafana. I am confident in my ability to learn and adapt to new challenges, and I believe my skills as a proactive and dedicated student make me a valuable candidate for a DevOps engineering role."

In [15]:
student.run("I'm looking for a Production engineer, do you have any experience?")



> Entering new AgentExecutor chain...
{
    "action": "qa-student",
    "action_input": "Do I have any experience as a Production engineer?"
}
Observation: Yes, you have experience as a Production Engineer. You participated in the Major League Hacking Fellowship as a Production Engineering Fellow, where you completed a 12-week curriculum-based learning program on core Production Engineering topics. As part of this fellowship, you developed a personal portfolio website using various technologies such as Python, Flask, Jinja, Tailwind CSS, MySQL, Nginx, Unittest, Docker, and the Google Maps API. You also gained experience in automated testing and deployment using CI/CD with GitHub Actions, and in setting up monitoring environments using Prometheus and Grafana.
Thought:{
    "action": "Final Answer",
    "action_input": "Yes, I do have experience as a Production Engineer. I participated in the Major League Hacking Fellowship as a Production Engineering Fellow, where I gained hands-on ex

'Yes, I do have experience as a Production Engineer. I participated in the Major League Hacking Fellowship as a Production Engineering Fellow, where I gained hands-on experience in core Production Engineering topics. I also developed a personal portfolio website using Python, Flask, Jinja, Tailwind CSS, MySQL, Nginx, Unittest, Docker, and the Google Maps API. Additionally, I have experience in automated testing and deployment using CI/CD with GitHub Actions, as well as setting up monitoring environments using Prometheus and Grafana.'

In [28]:
qa.run(promptTopic.format(topic="DevOps"))

Ruy Guzman Camacho has experience in DevOps through his Major League Hacking Fellowship and his volunteer work at IBM. Here is an overview of his DevOps experience:

1. Major League Hacking Fellowship (May 2022-Aug 2022):
   - Completed 12 weeks of curriculum-based learning on core Production Engineering topics.
   - Developed an open-source personal portfolio website using Python, Flask, Jinja, Tailwind CSS, MySQL, Nginx, Unittest, Docker, and the Google Maps API.
   - Automated testing and deployment using CI/CD with GitHub Actions.
   - Set up monitoring environments using Prometheus and Grafana.

2. IBM (Feb 2023 – Jun 2023):
   - Worked as a full-stack developer on a web dashboard aimed at managing certifications within IBM's employees.
   - Developed a web server using Ruby on Rails, connected to an Azure MySQL database.
   - Developed API tests using RSpec within the Rails project.
   - Set up testing and deployment using Docker and CI/CD with GitHub Actions.
   - Developed mult

In [37]:
qa.run(promptLanguage.format(language="Ruby"))

"Yes, Ruby is one of the programming languages that Ruy is familiar with. He has used Ruby on Rails to develop a web dashboard for IBM to manage certifications within the company's employees. The web server he developed was connected to an Azure MySQL database. Additionally, he developed API tests using RSpec within the Rails project."

In [38]:
qa.run(promptFramework.format(framework="React"))

'Yes, Ruy has experience using React. In his Major League Hacking Fellowship, he worked on a project called Geriatrik/Geriatrik-API, where he served as a full-stack developer. He developed authentication components for the application, defining endpoints on the API and implementing authorization middleware. This project involved using React as the front-end framework.\n\nAdditionally, in his Relevant Coursework section, Ruy mentions taking a course on Software Development, which included learning React.js. Although specific details about his experience with React.js in this course are not provided, it indicates that he has knowledge and understanding of the framework.'